WaveNet LSTM model without any decomposition (easy model for paper, performance is slightly less accurate than VWD-WaveNet-LSTM)

**Ensemble 1D-CNN, LSTM, GRU with Simple Average and Linear Regression Ensemble methods**

The goal is for prediction of arctic river under the background of climate change projections

The input includes ET, T, and P

PET value updated (from evspsbl) for correction and consistency

**Single step ahead forcasting**

It can:

1) using prcp, temp and disc to predict the next disc value
2) using prcp and temp to predict disc at the next time step
3) using prcp and temp to predict disc at the current time step by create a direct mapping between prcp, temp and disc

In [1]:
# Set up input step and lead time

N_input = 27  # Example: Using past 3 days of data
N_lead = 1  # Example: Predicting discharge 1 days into the future

In [2]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Conv1D, Add, Activation, Multiply, Flatten
from tensorflow.keras.callbacks import EarlyStopping

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU, SimpleRNN
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Activation, Dot, RepeatVector, Permute, Multiply, Lambda, concatenate
from tensorflow.keras import backend as K

In [3]:
import numpy as np
import pandas as pd
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer, MaxAbsScaler
from tensorflow.keras import layers, optimizers, metrics, Sequential, initializers
from tensorflow.keras.layers import (
    Dense,
    Dropout,
    BatchNormalization,
    LeakyReLU,
    Activation,
    Conv1D,
    MaxPooling1D,
    LSTM,
    GRU,
    SimpleRNN,
    RepeatVector,
    TimeDistributed,
    Flatten,
)

from tensorflow.keras.callbacks import (
    ModelCheckpoint,
    ReduceLROnPlateau,
    EarlyStopping,
)
from tensorflow.keras.regularizers import l2

from tensorflow.python import pywrap_tensorflow
import pickle
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
import mpl_toolkits.axisartist.axislines as axislines
from tensorflow import keras

from matplotlib.ticker import MultipleLocator, FormatStrFormatter, PercentFormatter
from tensorflow.keras.utils import plot_model
from scipy.stats import norm

from datetime import datetime as dt
import matplotlib.dates as mdates

# Import Data

In [4]:
# Import date from Github directly

import pandas as pd
from datetime import datetime

custom_date_parser = lambda x: datetime.strptime(x, "%m/%d/%Y")    # "%Y-%d-%m %H:%M:%S"

permalink = "https://github.com/Zhou-R/Dataset/blob/52f2bd7eda24623ac2c5aa678ed24d3463aae954/with%20Snow_Ob_Yenisei_Lena_Kolyma_Yukon_Mackenzie.csv"

additional_part = "?raw=true.csv"
url = permalink + additional_part

# url = "https://github.com/Zhou-R/Dataset/blob/8d5e4fc7e0522a90602e7b4aed6ab7aee030c70b/Ob_Yenisei_Lena.csv?raw=true.csv"

input_data_raw = pd.read_csv(url,parse_dates = ['Date'], date_parser = custom_date_parser)

# input_data_raw = pd.read_csv(url,parse_dates = ['Date'], date_parser = custom_date_parser)

# input_data_raw.dropna(inplace=True)
# input_data_raw.reset_index(drop=True, inplace=True)

input_data_raw


<ipython-input-4-0351d7bcdc35>:15: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  input_data_raw = pd.read_csv(url,parse_dates = ['Date'], date_parser = custom_date_parser)


,Date,SWE_Ob,SWE_Yenise,SWE_Lena,SWE_Kolyma,SWE_Yukon,SWE_Mackenzie,SM_Ob,SM_Yenise,SM_Lena,...,Kolyma_Precipitation (mm),Kolyma_Potential evapotranspiration (mm),Yukon_Temperature (C),Yukon_Observed discharge (mm),Yukon_Precipitation (mm),Yukon_Potential evapotranspiration (mm),Mackenzie_Temperature (C),Mackenzie_Observed discharge (mm),Mackenzie_Precipitation (mm),Mackenzie_Potential evapotranspiration (mm)
0,1936-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.125431,0.472144,-24.624243,NaN,7.804178,2.392983,-29.394011,NaN,17.635900,2.527454
1,1936-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.295411,0.383567,-27.527275,NaN,9.356810,3.408772,-31.045260,NaN,10.617746,3.877121
2,1936-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.523407,4.423247,-16.913876,NaN,15.041210,16.745934,-15.648836,NaN,14.087312,16.345925
3,1936-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.350601,16.947895,-6.478629,NaN,6.853509,38.267942,-7.571630,NaN,16.235885,37.320301
4,1936-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.810421,50.991586,3.549920,NaN,18.542920,70.983576,5.273794,NaN,23.493164,83.620051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039,2022-08-01,0.010251,1.641243,0.029106,0.009382,119.500792,0.575625,0.145323,1.128653,0.458078,...,NaN,58.657716,10.251515,40.870011,NaN,55.191868,13.789768,20.508288,NaN,84.860569
1040,2022-09-01,0.339322,14.711337,5.760139,7.463180,121.238436,1.002507,0.495607,5.470934,7.212832,...,NaN,29.903809,5.092185,40.964218,NaN,30.387559,6.944426,14.455632,NaN,43.148091
1041,2022-10-01,2.705581,32.280331,31.442557,50.313798,138.880708,9.130506,5.153197,11.761342,4.207534,...,NaN,6.982766,-4.030303,36.306633,NaN,10.007018,-0.101914,13.009968,NaN,19.822462
1042,2022-11-01,23.505568,29.224834,59.904832,77.908022,169.485242,32.188336,7.344044,0.564037,0.030927,...,NaN,0.000000,-14.533973,19.330699,NaN,2.411483,-13.472463,7.002528,NaN,4.188020


In [5]:
# select certain columns and create a new dataset with data we need
raw_date = input_data_raw.Date
raw_disc = input_data_raw["Yenisei_Observed discharge (mm)"]
raw_temp = input_data_raw["Yenisei_Temperature (C)"]
raw_prcp = input_data_raw["Yenisei_Precipitation (mm)"]
raw_evap = input_data_raw["Yenisei_Potential evapotranspiration (mm)"]
raw_sm = input_data_raw["SM_Yenise"]
raw_swe = input_data_raw["SWE_Yenise"]

dataset = pd.concat([raw_date, raw_disc, raw_temp, raw_prcp, raw_evap,raw_sm,raw_swe], axis=1)

dataset.dropna(inplace=True)
dataset.reset_index(drop=True, inplace=True)

print(dataset.shape)

# input_data = dataset.reset_index(drop=True)

# create for input
dataset_date = dataset.Date
dataset_disc = dataset["Yenisei_Observed discharge (mm)"]
dataset_temp = dataset["Yenisei_Temperature (C)"]
dataset_prcp = dataset["Yenisei_Precipitation (mm)"]
dataset_evap = dataset["Yenisei_Potential evapotranspiration (mm)"]
dataset_sm = dataset["SM_Yenise"]
dataset_swe = dataset["SWE_Yenise"]

(767, 7)


In [6]:
# # --- Process prcp with VMD ---
# !pip install vmdpy
# from vmdpy import VMD

# # Initialize VMD parameters
# alpha = 2000       # moderate bandwidth constraint
# tau = 0.            # noise-tolerance (no strict fidelity enforcement)
# numb_K = 5              # 4 modes
# DC = 0             # no DC part imposed
# init = 1           # initialize omegas uniformly
# tol = 1e-6

# # Perform VMD
# u, u_hat, omega = VMD(dataset_prcp, alpha, tau, numb_K, DC, init, tol)

# # turn array into dataframe
# dataset_prcp = pd.DataFrame(u.T)

# # for i in range(K):
# #     plt.subplot(K, 1, i + 1)
# #     plt.plot(u[i])
# #     plt.title(f'Mode {i+1}')

# # plt.tight_layout()
# # plt.show()

In [7]:
# Normalize data using MinMaxScaler
scaler_X_prcp = MinMaxScaler()
scaler_X_temp = MinMaxScaler()
scaler_X_evap = MinMaxScaler()
scaler_X_sm = MinMaxScaler()
scaler_X_swe = MinMaxScaler()

prcp_normalized = scaler_X_prcp.fit_transform(dataset_prcp.values.reshape(-1, 1))
temp_normalized = scaler_X_temp.fit_transform(dataset_temp.values.reshape(-1, 1))
evap_normalized = scaler_X_evap.fit_transform(dataset_evap.values.reshape(-1, 1))
sm_normalized = scaler_X_sm.fit_transform(dataset_sm.values.reshape(-1, 1))
swe_normalized = scaler_X_swe.fit_transform(dataset_swe.values.reshape(-1, 1))

scaler_Y = MinMaxScaler()
disc_normalized = scaler_Y.fit_transform(dataset_disc.values.reshape(-1, 1))


In [8]:
def generate_sequences(prcp, temp, evap, sm, swe, disc, sequence_length, lead_time):
    X, Y = [], []
    for i in range(sequence_length, len(prcp) - lead_time + 1):
        # input features
        X_prec = prcp[i-sequence_length:i]
        X_temp = temp[i-sequence_length:i]
        X_evap = evap[i-sequence_length:i]
        X_sm = sm[i-sequence_length:i]
        X_swe = swe[i-sequence_length:i]

        X_disc = disc[i-sequence_length:i]   # its own historical data as input as well
        # X_combined = np.column_stack((X_prec, X_temp, X_disc)) # include its own historical discharge.

        # X_combined = np.column_stack((X_prec, X_temp, X_evap, X_sm, X_swe)) # does not include its own historical discharge
        X_combined = np.column_stack((X_prec, X_temp, X_evap)) # does not include its own historical discharge

        # output variable and combination
        X.append(X_combined)
        Y.append(disc[i + lead_time - 1])   # Add discharge of next time step
        # Y.append(disc[i + lead_time - 1 - 1])   # Add discharge of current time step

    return np.array(X), np.array(Y)

sequence_length = N_input  # Example: Using past several days of data
lead_time = N_lead  # Example: Predicting discharge 1 days into the future

X, Y = generate_sequences(prcp_normalized, temp_normalized, evap_normalized, sm_normalized, swe_normalized, disc_normalized, sequence_length, lead_time)


In [9]:
# Split the data into training and testing sets (80% train, 20% test)
full_length = len(disc_normalized)

train_size = int(full_length * 0.7)
val_size = int(full_length * 0.15)

X_train, X_val, X_test = X[:train_size], X[train_size:train_size+val_size], X[train_size+val_size:]
Y_train, Y_val, Y_test = Y[:train_size], Y[train_size:train_size+val_size], Y[train_size+val_size:]

In [10]:
# evaluate validation dataset with RMSE, MAE_error, NSE
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import mean_absolute_error as mae

def evaluate_forecasts(result_a, result_p):
    MSE = np.square(np.subtract(result_p,result_a)).mean()
    RMSE = MSE**(1/2)
    MAE_error = mae(result_a, result_p)

    def nse(predictions, targets):
        return (1-(np.sum((predictions-targets)**2)/np.sum((targets-np.mean(targets))**2)))
    NSE = nse(result_p, result_a)
    return [RMSE, MAE_error,NSE]

## DL model

### LSTM

In [11]:
# Model Parameters
n_units = 64  # Number of LSTM units (you can adjust this)

input_shape = (X_train.shape[1], X_train.shape[2])

# LSTM Model Definition
model_lstm = Sequential()
model_lstm.add(LSTM(64, input_shape=input_shape, return_sequences=True))
model_lstm.add(Dropout(0.4))  # Adding dropout for regularization
model_lstm.add(LSTM(32))
model_lstm.add(Dropout(0.4))
model_lstm.add(Dense(1))  # Output layer for regression

early_stopping = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, restore_best_weights=True)

# Compile the model
optimizer = Adam(learning_rate=0.005)
model_lstm.compile(optimizer=optimizer, loss='mean_squared_error')
history_lstm = model_lstm.fit(X_train, Y_train, epochs=150, batch_size=32, validation_data=(X_val, Y_val), callbacks=[early_stopping])


Epoch 1/150
17/17 [==============================] - 19s 196ms/step - loss: 0.0550 - val_loss: 0.0251
Epoch 2/150
17/17 [==============================] - 1s 62ms/step - loss: 0.0385 - val_loss: 0.0223
Epoch 3/150
17/17 [==============================] - 1s 57ms/step - loss: 0.0354 - val_loss: 0.0205
Epoch 4/150
17/17 [==============================] - 1s 31ms/step - loss: 0.0331 - val_loss: 0.0151
Epoch 5/150
17/17 [==============================] - 1s 31ms/step - loss: 0.0293 - val_loss: 0.0147
Epoch 6/150
17/17 [==============================] - 1s 31ms/step - loss: 0.0294 - val_loss: 0.0142
Epoch 7/150
17/17 [==============================] - 0s 29ms/step - loss: 0.0271 - val_loss: 0.0134
Epoch 8/150
17/17 [==============================] - 1s 31ms/step - loss: 0.0262 - val_loss: 0.0117
Epoch 9/150
17/17 [==============================] - 1s 33ms/step - loss: 0.0234 - val_loss: 0.0063
Epoch 10/150
17/17 [==============================] - 1s 36ms/step - loss: 0.0199 - val_loss: 0.00

In [12]:
# LSTM prediction
Y_pred_train_lstm_norm = model_lstm.predict(X_train)
Y_pred_val_lstm_norm = model_lstm.predict(X_val)
Y_pred_test_lstm_norm = model_lstm.predict(X_test)

# result of LSTM simulation
Y_pred_train_lstm = scaler_Y.inverse_transform(Y_pred_train_lstm_norm.reshape(-1, 1))
Y_pred_val_lstm = scaler_Y.inverse_transform(Y_pred_val_lstm_norm.reshape(-1, 1))
Y_pred_test_lstm = scaler_Y.inverse_transform(Y_pred_test_lstm_norm.reshape(-1, 1))

# observed data
Y_train_observe = scaler_Y.inverse_transform(Y_train.reshape(-1, 1))
Y_test_observe = scaler_Y.inverse_transform(Y_test.reshape(-1, 1))

3/3 [==============================] - 0s 11ms/step


In [13]:
# Evaluate the results

[RMSE, MAE_error, NSE] = evaluate_forecasts(Y_pred_test_lstm, Y_test_observe)

test_eval_metrics = pd.DataFrame()
test_eval_metrics[f'LSTM_in{sequence_length}_lead{lead_time}'] = [RMSE, MAE_error, NSE]

print("in testing dataset, for LSTM model:\n", "RMSE=", RMSE, "MAE=", MAE_error, "NSE=", NSE)

in testing dataset, for LSTM model:
 RMSE= 7.865497823445629 MAE= 5.6337778685590765 NSE= 0.6968616441731752


### GRU

In [14]:
# GRU Model Definition
model_gru = Sequential()
model_gru.add(GRU(64, input_shape=input_shape, return_sequences=True))
model_gru.add(Dropout(0.4))  # Adding dropout for regularization
model_gru.add(GRU(32))
model_gru.add(Dropout(0.4))
model_gru.add(Dense(1))  # Output layer for regression

early_stopping = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, restore_best_weights=True)

# Compile the model
optimizer = Adam(learning_rate=0.005)
model_gru.compile(optimizer=optimizer, loss='mean_squared_error')
history_gru = model_gru.fit(X_train, Y_train, epochs=150, batch_size=32, validation_data=(X_val, Y_val), callbacks=[early_stopping])


Epoch 1/150
17/17 [==============================] - 5s 84ms/step - loss: 0.0557 - val_loss: 0.0231
Epoch 2/150
17/17 [==============================] - 1s 34ms/step - loss: 0.0335 - val_loss: 0.0159
Epoch 3/150
17/17 [==============================] - 1s 50ms/step - loss: 0.0302 - val_loss: 0.0141
Epoch 4/150
17/17 [==============================] - 1s 50ms/step - loss: 0.0278 - val_loss: 0.0132
Epoch 5/150
17/17 [==============================] - 1s 52ms/step - loss: 0.0278 - val_loss: 0.0128
Epoch 6/150
17/17 [==============================] - 1s 50ms/step - loss: 0.0252 - val_loss: 0.0124
Epoch 7/150
17/17 [==============================] - 1s 52ms/step - loss: 0.0262 - val_loss: 0.0116
Epoch 8/150
17/17 [==============================] - 1s 39ms/step - loss: 0.0240 - val_loss: 0.0111
Epoch 9/150
17/17 [==============================] - 1s 34ms/step - loss: 0.0253 - val_loss: 0.0113
Epoch 10/150
17/17 [==============================] - 1s 35ms/step - loss: 0.0236 - val_loss: 0.0090

In [15]:
# GRU prediction
Y_pred_train_gru_norm = model_gru.predict(X_train)
Y_pred_val_gru_norm = model_gru.predict(X_val)
Y_pred_test_gru_norm = model_gru.predict(X_test)

# result of GRU simulation
Y_pred_train_gru = scaler_Y.inverse_transform(Y_pred_train_gru_norm.reshape(-1, 1))
Y_pred_val_gru = scaler_Y.inverse_transform(Y_pred_val_gru_norm.reshape(-1, 1))
Y_pred_test_gru = scaler_Y.inverse_transform(Y_pred_test_gru_norm.reshape(-1, 1))

3/3 [==============================] - 0s 8ms/step


In [16]:
[RMSE, MAE_error, NSE] = evaluate_forecasts(Y_pred_test_gru, Y_test_observe)

test_eval_metrics[f'GRU_in{sequence_length}_lead{lead_time}'] = [RMSE, MAE_error, NSE]

print("in testing dataset, for GRU model:\n", "RMSE=", RMSE, "MAE=", MAE_error, "NSE=", NSE)

in testing dataset, for GRU model:
 RMSE= 7.641360475953587 MAE= 5.273450414052556 NSE= 0.7459707079895512


### 1D-CNN

In [17]:
# CNN
model_cnn = Sequential()
model_cnn.add(Conv1D(filters=n_units, kernel_size=2, activation='relu', input_shape=input_shape))
# can add another cnn
# model_cnn.add(Dropout(0.4))
# model_cnn.add(Conv1D(filters=n_units, kernel_size=2, activation='relu'))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Flatten())
# model_cnn.add(layers.RepeatVector(n_output)) # not sure how RepeatVector, it should be the feature size of result, here is one https://github.com/zherbz/EncoderDecoder/blob/master/ResCNN_LSTM.py
model_cnn.add(Dense(50, activation='relu'))
model_cnn.add(Dense(1))
model_cnn.compile(optimizer='adam', loss='mse')

cnn_history = model_cnn.fit(X_train, Y_train, epochs=50, batch_size=32, validation_data=(X_val, Y_val), callbacks=[early_stopping])


Epoch 1/50
17/17 [==============================] - 1s 16ms/step - loss: 0.0275 - val_loss: 0.0104
Epoch 2/50
17/17 [==============================] - 0s 6ms/step - loss: 0.0209 - val_loss: 0.0094
Epoch 3/50
17/17 [==============================] - 0s 7ms/step - loss: 0.0183 - val_loss: 0.0074
Epoch 4/50
17/17 [==============================] - 0s 6ms/step - loss: 0.0170 - val_loss: 0.0070
Epoch 5/50
17/17 [==============================] - 0s 7ms/step - loss: 0.0155 - val_loss: 0.0077
Epoch 6/50
17/17 [==============================] - 0s 6ms/step - loss: 0.0153 - val_loss: 0.0091
Epoch 7/50
17/17 [==============================] - 0s 8ms/step - loss: 0.0144 - val_loss: 0.0069
Epoch 8/50
17/17 [==============================] - 0s 8ms/step - loss: 0.0129 - val_loss: 0.0073
Epoch 9/50
17/17 [==============================] - 0s 7ms/step - loss: 0.0123 - val_loss: 0.0064
Epoch 10/50
17/17 [==============================] - 0s 7ms/step - loss: 0.0127 - val_loss: 0.0061
Epoch 11/50
17/17 

In [18]:
# CNN prediction
Y_pred_train_cnn_norm = model_cnn.predict(X_train)
Y_pred_val_cnn_norm = model_cnn.predict(X_val)
Y_pred_test_cnn_norm = model_cnn.predict(X_test)

# result of cnn simulation
Y_pred_train_cnn = scaler_Y.inverse_transform(Y_pred_train_cnn_norm.reshape(-1, 1))
Y_pred_val_cnn = scaler_Y.inverse_transform(Y_pred_val_cnn_norm.reshape(-1, 1))
Y_pred_test_cnn = scaler_Y.inverse_transform(Y_pred_test_cnn_norm.reshape(-1, 1))

3/3 [==============================] - 0s 4ms/step


In [19]:
[RMSE, MAE_error, NSE] = evaluate_forecasts(Y_pred_test_cnn, Y_test_observe)

test_eval_metrics[f'CNN_in{sequence_length}_lead{lead_time}'] = [RMSE, MAE_error, NSE]

print("in testing dataset, for 1D-CNN model:\n", "RMSE=", RMSE, "MAE=", MAE_error, "NSE=", NSE)

in testing dataset, for 1D-CNN model:
 RMSE= 8.037448142360923 MAE= 5.012576168373056 NSE= 0.8084393832283913


# Ensemble model - Linear


### Simple Average

In [20]:
# Simple averaging ensemble
simple_ensemble_pred = (Y_pred_test_lstm + Y_pred_test_gru + Y_pred_test_cnn) / 3

[RMSE, MAE_error, NSE] = evaluate_forecasts(Y_test_observe, simple_ensemble_pred)

test_eval_metrics[f"Ensem-SAM_in{sequence_length}_lead{lead_time}"] = [RMSE, MAE_error, NSE]

print("For Simple averaging ensemble model:\n", "RMSE=", RMSE, "MAE=", MAE_error, "NSE=", NSE)

For Simple averaging ensemble model:
 RMSE= 7.246251105097012 MAE= 4.736632789499964 NSE= 0.8234465898600314


### LinearRegression

In [21]:
from sklearn.linear_model import LinearRegression

# 1. Stack the predictions together to use as new features
train_preds = np.column_stack((Y_pred_train_lstm, Y_pred_train_gru, Y_pred_train_cnn))
test_preds = np.column_stack((Y_pred_test_lstm, Y_pred_test_gru, Y_pred_test_cnn))

# 2. Train a second-level model on these new features (fit the observed data in the training set: labels_train)
linear_meta_model = LinearRegression()
linear_meta_model.fit(train_preds, Y_train_observe)

# The coefficients assigned by the meta_model are the weights for the ensemble.
linear_weights = linear_meta_model.coef_
print(linear_weights)

# 3. Use the second-level model to make a final prediction on validation data
liear_final_pred = linear_meta_model.predict(test_preds)


# ## alternative way
# liear_final_pred_plot = linear_weights[0][0]*label_vali_simul_lstm_plot + linear_weights[0][1]*label_vali_simul_gru_plot + linear_weights[0][2]*label_vali_simul_cnn_plot

[[-0.5751122  0.4863627  1.1265146]]


In [22]:
[RMSE, MAE_error, NSE] = evaluate_forecasts(Y_test_observe, liear_final_pred)

test_eval_metrics[f'Ensem-LR_in{sequence_length}_lead{lead_time}'] = [RMSE, MAE_error, NSE]


print("For stacking ensemble model with LinearRegression:\n", "RMSE=", RMSE, "MAE=", MAE_error, "NSE=", NSE)

For stacking ensemble model with LinearRegression:
 RMSE= 8.494362002328883 MAE= 5.55579741966419 NSE= 0.7573887799979251


## Evaluation Metrics

In [23]:
# import matplotlib.pyplot as plt
# import numpy as np

# plt.figure(figsize=(16, 9))

# x = dataset_date[train_size+val_size+sequence_length+lead_time-1:]
# # Plot the two curves
# plt.plot(x, Y_test_final, 'r-', label='Predicted discharge')
# # plt.plot(x, Y_pred_test_simu_first, 'g-', label='1st LSTM Predicted discharge')
# plt.plot(x, Y_test_observe, 'b--', label='Observed discharge')


# # Add a title and axis labels
# plt.xlabel('Date')
# plt.ylabel('Discharge $m^3/s$')

# # Add a legend
# plt.legend()

# # Show the plot

In [24]:
new_index = ["RMSE", "MAE", "NSE"]

test_eval_metrics.index = new_index

test_eval_metrics

,LSTM_in27_lead1,GRU_in27_lead1,CNN_in27_lead1,Ensem-SAM_in27_lead1,Ensem-LR_in27_lead1
RMSE,7.865498,7.641360,8.037448,7.246251,8.494362
MAE,5.633778,5.273450,5.012576,4.736633,5.555797
NSE,0.696862,0.745971,0.808439,0.823447,0.757389


## Export the results

In [25]:
# # Prediction Results

# # date_output = xtick_date.values.astype('float64')
# data_output = x.reset_index(drop=True)

# predits = pd.concat([pd.DataFrame(data_output), pd.DataFrame(Y_test_observe.ravel()),  pd.DataFrame(Y_pred_test_lstm.ravel()), pd.DataFrame(Y_pred_test_gru.ravel()), pd.DataFrame(Y_pred_test_cnn.ravel()),pd.DataFrame(simple_ensemble_pred.ravel()),pd.DataFrame(liear_final_pred.ravel())], axis=1)

# predits.columns = ['Date', 'Observed discharge', 'LSTM', 'GRU', '1D-CNN','Ensem-SAM','Ensem-LR' ]

In [26]:
# download the evaluation metrics for training dataset

# train_metrics_name = f'Ensemble_train_eval_in{sequence_length}_out{sequence_length}_lead{lead_time}.xlsx'
# train_eval_metrics.to_excel(train_metrics_name, index=True)

# # After saving the file as described above
# files.download(train_metrics_name)

In [27]:
# from google.colab import files

# test_metrics_name = f'Ensemble_test_eval_in{sequence_length}_out{sequence_length}_lead{lead_time}.xlsx'
# test_eval_metrics.to_excel(test_metrics_name, index=True)

# # After saving the file as described above
# files.download(test_metrics_name)

In [28]:
# # download the prediction results
# predits_name = f'Ensemble_prediction_in{sequence_length}_out{sequence_length}_lead{lead_time}.xlsx'
# predits.to_excel(predits_name, index=False)

# # After saving the file as described above
# files.download(predits_name)

# All in one For loop for all climate projections

In [29]:
# Input Climate Date

custom_date_parser = lambda x: datetime.strptime(x, "%m/%d/%Y")    # "%Y-%d-%m %H:%M:%S"

# permalink = "https://github.com/Zhou-R/Dataset/blob/fdbb3e05cc31c1a0b84958ea4834bdc3c3602a69/Climate%20Projection%20AIO_final_PETupdate.csv"
permalink = "https://github.com/Zhou-R/Dataset/blob/e480dabbfe9dde15dbbacc39fa69f4e9b8747d6b/Climate%20Projection%20AIO_corrected.csv"
additional_part = "?raw=true.csv"
url = permalink + additional_part

# Run and Download all climate models
temp_data = "-tas"
prcp_data = "-pr"
evap_data = "-pet"
sm_data = "-snm"
swe_data = "-snw"
location = "-Yenisei"

climate_models = [
    "BCC-CSM2-MR-SSP126", "BCC-CSM2-MR-SSP245", "BCC-CSM2-MR-SSP370", "BCC-CSM2-MR-SSP585",
    "CanESM5-SSP126", "CanESM5-SSP245", "CanESM5-SSP370", "CanESM5-SSP585",
    "IPSL-CM6A-LR-SSP126", "IPSL-CM6A-LR-SSP245", "IPSL-CM6A-LR-SSP370", "IPSL-CM6A-LR-SSP585",
    "MPI-ESM1-2-HR-SSP126", "MPI-ESM1-2-HR-SSP245", "MPI-ESM1-2-HR-SSP370", "MPI-ESM1-2-HR-SSP585"
]

def generate_input_sequences(prcp, temp, evap, sm, swe, sequence_length, lead_time):
    X = []
    for i in range(sequence_length, len(prcp) - lead_time + 1):
        # input features
        X_prec = prcp[i-sequence_length:i]
        X_temp = temp[i-sequence_length:i]
        X_evap = evap[i-sequence_length:i]
        X_sm = sm[i-sequence_length:i]
        X_swe = swe[i-sequence_length:i]
        # X_combined = np.column_stack((X_prec, X_temp, X_disc)) # include its own historical discharge.

        X_combined = np.column_stack((X_prec, X_temp, X_evap)) # does not include its own historical discharge
        # X_combined = np.column_stack((X_prec, X_temp, X_evap, X_sm, X_swe)) # does not include its own historical discharge

        # output variable and combination
        X.append(X_combined)
        # Y.append(disc[i + lead_time - 1 - 1])   # Add discharge of current time step

    return np.array(X)

from google.colab import files

# Initialize a DataFrame to store all predictions
all_predictions = pd.DataFrame()
all_predictions_lstm = pd.DataFrame()
all_predictions_gru = pd.DataFrame()
all_predictions_cnn = pd.DataFrame()
all_predictions_sam = pd.DataFrame()
all_predictions_lr = pd.DataFrame()

for climate_model in climate_models:

    project_data_raw = pd.read_csv(url,parse_dates = ['Date'], date_parser = custom_date_parser)
    # select certain columns and create a new dataset with data we need
    project_date = project_data_raw.Date
    project_temp = project_data_raw[climate_model+temp_data+location]
    project_prcp = project_data_raw[climate_model+prcp_data+location]
    project_evap = project_data_raw[climate_model+evap_data+location]
    project_sm = project_data_raw[climate_model+sm_data+location]
    project_swe = project_data_raw[climate_model+swe_data+location]

    project_dataset = pd.concat([project_date, project_temp, project_prcp, project_evap, project_sm, project_swe], axis=1)

    project_dataset.dropna(inplace=True)
    project_dataset.reset_index(drop=True, inplace=True)

    # input_data = dataset.reset_index(drop=True)

    # create for input
    project_dataset_date = project_dataset.Date
    project_dataset_temp = project_dataset[climate_model+temp_data+location]
    project_dataset_prcp = project_dataset[climate_model+prcp_data+location]
    project_dataset_evap = project_dataset[climate_model+evap_data+location]
    project_dataset_sm = project_dataset[climate_model+sm_data+location]
    project_dataset_swe = project_dataset[climate_model+swe_data+location]

    # # Perform VMD
    # u_proj, u_hat_proj, omega_proj = VMD(project_dataset_prcp, alpha, tau, numb_K, DC, init, tol)

    # # turn array into dataframe
    # project_dataset_prcp = pd.DataFrame(u_proj.T)

    # Normalize the input and generate the input sequence for prediction
    project_prcp_normalized = scaler_X_prcp.fit_transform(project_dataset_prcp.values.reshape(-1, 1))
    project_temp_normalized = scaler_X_temp.fit_transform(project_dataset_temp.values.reshape(-1, 1))
    project_evap_normalized = scaler_X_evap.fit_transform(project_dataset_evap.values.reshape(-1, 1))
    project_sm_normalized = scaler_X_sm.fit_transform(project_dataset_sm.values.reshape(-1, 1))
    project_swe_normalized = scaler_X_swe.fit_transform(project_dataset_swe.values.reshape(-1, 1))

    X_proj = generate_input_sequences(project_prcp_normalized, project_temp_normalized, project_evap_normalized, project_sm_normalized, project_swe_normalized, sequence_length, lead_time)

    # Make final predictions (two combined)

    # LSTM Model
    Y_pred_proj_lstm = model_lstm.predict(X_proj)
    # GRU Model
    Y_pred_proj_gru = model_gru.predict(X_proj)
    # CNN Model
    Y_pred_proj_cnn = model_cnn.predict(X_proj)

    # Inverse the normalization
    Y_final_proj_lstm = scaler_Y.inverse_transform(Y_pred_proj_lstm.reshape(-1, 1))
    Y_final_proj_gru = scaler_Y.inverse_transform(Y_pred_proj_gru.reshape(-1, 1))
    Y_final_proj_cnn = scaler_Y.inverse_transform(Y_pred_proj_cnn.reshape(-1, 1))

    Y_final_proj_sam = (Y_final_proj_lstm + Y_final_proj_gru + Y_final_proj_cnn) / 3

    # --- Ensemble - Linear Regression
    pro_preds = np.column_stack((Y_final_proj_lstm, Y_final_proj_gru, Y_final_proj_cnn))
    Y_final_proj_lr = linear_meta_model.predict(pro_preds)

    # ---- download each climate model together ----

    # Combine the prediction results with dates
    x_proj = project_dataset_date[sequence_length+lead_time-1:]
    project_data_output = x_proj.reset_index(drop=True)

    project_predits_lstm = pd.DataFrame(Y_final_proj_lstm.ravel(), columns=[climate_model])
    project_predits_gru = pd.DataFrame(Y_final_proj_gru.ravel(), columns=[climate_model])
    project_predits_cnn = pd.DataFrame(Y_final_proj_cnn.ravel(), columns=[climate_model])

    project_predits_sam = pd.DataFrame(Y_final_proj_sam.ravel(), columns=[climate_model])
    project_predits_lr = pd.DataFrame(Y_final_proj_lr.ravel(), columns=[climate_model])


    # --- LSTM ------
    if all_predictions_lstm.empty:
        all_predictions_lstm = pd.DataFrame(project_data_output)
        all_predictions_lstm = all_predictions_lstm.join(project_predits_lstm)
    else:
        # For subsequent models, just add the predictions as new columns
        all_predictions_lstm = all_predictions_lstm.join(project_predits_lstm)

    # ----- GRU ------
    if all_predictions_gru.empty:
        all_predictions_gru = pd.DataFrame(project_data_output)
        all_predictions_gru = all_predictions_gru.join(project_predits_gru)
    else:
        # For subsequent models, just add the predictions as new columns
        all_predictions_gru = all_predictions_gru.join(project_predits_gru)

    # ------CNN ------
    if all_predictions_cnn.empty:
        all_predictions_cnn = pd.DataFrame(project_data_output)
        all_predictions_cnn = all_predictions_cnn.join(project_predits_cnn)
    else:
        # For subsequent models, just add the predictions as new columns
        all_predictions_cnn = all_predictions_cnn.join(project_predits_cnn)

    # ------ Ensemble-SAM ------
    if all_predictions_sam.empty:
        all_predictions_sam = pd.DataFrame(project_data_output)
        all_predictions_sam = all_predictions_sam.join(project_predits_sam)
    else:
        # For subsequent models, just add the predictions as new columns
        all_predictions_sam = all_predictions_sam.join(project_predits_sam)

    # ------ Ensemble-LR ------
    if all_predictions_lr.empty:
        all_predictions_lr = pd.DataFrame(project_data_output)
        all_predictions_lr = all_predictions_lr.join(project_predits_lr)
    else:
        # For subsequent models, just add the predictions as new columns
        all_predictions_lr = all_predictions_lr.join(project_predits_lr)


<ipython-input-29-07f05977f3a1>:57: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  project_data_raw = pd.read_csv(url,parse_dates = ['Date'], date_parser = custom_date_parser)


32/32 [==============================] - 0s 3ms/step


<ipython-input-29-07f05977f3a1>:57: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  project_data_raw = pd.read_csv(url,parse_dates = ['Date'], date_parser = custom_date_parser)


32/32 [==============================] - 0s 2ms/step


<ipython-input-29-07f05977f3a1>:57: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  project_data_raw = pd.read_csv(url,parse_dates = ['Date'], date_parser = custom_date_parser)


...

32/32 [==============================] - 0s 2ms/step


<ipython-input-29-07f05977f3a1>:57: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  project_data_raw = pd.read_csv(url,parse_dates = ['Date'], date_parser = custom_date_parser)


32/32 [==============================] - 0s 2ms/step


<ipython-input-29-07f05977f3a1>:57: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  project_data_raw = pd.read_csv(url,parse_dates = ['Date'], date_parser = custom_date_parser)


32/32 [==============================] - 0s 2ms/step


In [30]:
# Download monthly projection

# ---- Save LSTM--- Save the combined predictions to an Excel file
all_predictions_lstm_name = f'{location}_lstm_monthly_climate_models.xlsx'
all_predictions_lstm.to_excel(all_predictions_lstm_name, index=False)
files.download(all_predictions_lstm_name)

# ---- Save GRU--- Save the combined predictions to an Excel file
all_predictions_gru_name = f'{location}_gru_monthly_climate_models.xlsx'
all_predictions_gru.to_excel(all_predictions_gru_name, index=False)
files.download(all_predictions_gru_name)

# ---- Save CNN--- Save the combined predictions to an Excel file
all_predictions_cnn_name = f'{location}_cnn_monthly_climate_models.xlsx'
all_predictions_cnn.to_excel(all_predictions_cnn_name, index=False)
files.download(all_predictions_cnn_name)

# ---- Save Ensemble-SAM --- Save the combined predictions to an Excel file
all_predictions_sam_name = f'{location}_sam_monthly_climate_models.xlsx'
all_predictions_sam.to_excel(all_predictions_sam_name, index=False)
files.download(all_predictions_sam_name)

# ---- Save Ensemble LR --- Save the combined predictions to an Excel file
all_predictions_lr_name = f'{location}_lr_monthly_climate_models.xlsx'
all_predictions_lr.to_excel(all_predictions_lr_name, index=False)
files.download(all_predictions_lr_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
# Convert to annual dishcarge
all_predictions_lstm['Date'] = pd.to_datetime(all_predictions_lstm['Date'])
all_predictions_lstm.set_index('Date', inplace=True)
all_predictions_lstm_annual = all_predictions_lstm.resample('Y').sum()

all_predictions_gru['Date'] = pd.to_datetime(all_predictions_gru['Date'])
all_predictions_gru.set_index('Date', inplace=True)
all_predictions_gru_annual = all_predictions_gru.resample('Y').sum()

all_predictions_cnn['Date'] = pd.to_datetime(all_predictions_cnn['Date'])
all_predictions_cnn.set_index('Date', inplace=True)
all_predictions_cnn_annual = all_predictions_cnn.resample('Y').sum()

all_predictions_sam['Date'] = pd.to_datetime(all_predictions_sam['Date'])
all_predictions_sam.set_index('Date', inplace=True)
all_predictions_sam_annual = all_predictions_sam.resample('Y').sum()

all_predictions_lr['Date'] = pd.to_datetime(all_predictions_lr['Date'])
all_predictions_lr.set_index('Date', inplace=True)
all_predictions_lr_annual = all_predictions_lr.resample('Y').sum()

# ---- Save LSTM--- Save the combined predictions to an Excel file
all_predictions_lstm_annual_name = f'{location}_lstm_annual_climate_models.xlsx'
all_predictions_lstm_annual.to_excel(all_predictions_lstm_annual_name, index=True)
files.download(all_predictions_lstm_annual_name)

# ---- Save GRU--- Save the combined predictions to an Excel file
all_predictions_gru_annual_name = f'{location}_gru_annual_climate_models.xlsx'
all_predictions_gru_annual.to_excel(all_predictions_gru_annual_name, index=True)
files.download(all_predictions_gru_annual_name)

# ---- Save CNN--- Save the combined predictions to an Excel file
all_predictions_cnn_annual_name = f'{location}_cnn_annual_climate_models.xlsx'
all_predictions_cnn_annual.to_excel(all_predictions_cnn_annual_name, index=True)
files.download(all_predictions_cnn_annual_name)

# ---- Save Ensemble-SAM --- Save the combined predictions to an Excel file
all_predictions_sam_annual_name = f'{location}_sam_annual_climate_models.xlsx'
all_predictions_sam_annual.to_excel(all_predictions_sam_annual_name, index=True)
files.download(all_predictions_sam_annual_name)

# ---- Save Ensemble-LR --- Save the combined predictions to an Excel file
all_predictions_lr_annual_name = f'{location}_lr_annual_climate_models.xlsx'
all_predictions_lr_annual.to_excel(all_predictions_lr_annual_name, index=True)
files.download(all_predictions_lr_annual_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>